In [1]:
import time
iss = 0

def make_submit(pred):
    global iss
    submit = pd.DataFrame()
    submit['id'] = test.index
    submit['skilled'] = pred
    iss += 1
    submit.to_csv('submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss), index=False)
    print('last: ', 'submissions/submission{}-{}.csv'.format(''.join(time.ctime().split()[2]), iss))

In [2]:
import matplotlib.pyplot as plt
def print_f_imp(clf,n_top=10):
    feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
    feat_imp['feature'] = train.columns
    feat_imp.sort_values(by='importance', ascending=False, inplace=True)
    feat_imp_r = feat_imp
    feat_imp = feat_imp.iloc[:n_top]

    feat_imp.sort_values(by='importance', inplace=True)
    feat_imp = feat_imp.set_index('feature', drop=True)
    feat_imp.plot.barh(figsize=(8,8))
    
    plt.xlabel('Feature Importance Score')
    plt.show()
    return  feat_imp_r

In [3]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import lightgbm
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import catboost 
import xgboost

In [22]:
train = pd.read_csv('data/trian_v3_1.csv', index_col='id')
test = pd.read_csv('data/test_v3_1.csv', index_col='id')
target = train.skilled

In [26]:
train_id_damage_targets = pd.read_csv('json_data/train_id_damage_targets.csv', index_col='Unnamed: 0')

In [27]:
dtc = train_id_damage_targets.columns.tolist()

In [28]:
neutral = []
badguys = []
goodguys = []
for i  in dtc:
    a = i.split('_')
    if 'neutral' in a:
        neutral.append(i)
    if 'badguys' in a:
        badguys.append(i)
    if 'goodguys' in a:
        goodguys.append(i)

In [29]:
train['dmt_goodguys'] = train[goodguys].sum(axis=1)
train['dmt_badguys'] = train[badguys].sum(axis=1)
train['dmt_neutral'] = train[neutral].sum(axis=1)

In [30]:
test['dmt_goodguys'] = test[goodguys].sum(axis=1)
test['dmt_badguys'] = test[badguys].sum(axis=1)
test['dmt_neutral'] = test[neutral].sum(axis=1)

In [31]:
train = train.drop(['skilled'], axis=1)

In [33]:
X = train[test.columns]
x_test = test
y = target

In [34]:
k_fold = StratifiedKFold(n_splits=5, shuffle=True)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=22)

In [36]:
clf =  lightgbm.LGBMClassifier(n_estimators=1500, learning_rate=0.1, max_depth=12, reg_alpha=9, min_child_weight=8, n_jobs=-1, random_state=1) #@- 82 #reg_alpha=10

In [37]:
%%time
clf.fit(X_train, y_train)

CPU times: user 8min 25s, sys: 6.58 s, total: 8min 31s
Wall time: 1min 6s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=12,
        min_child_samples=20, min_child_weight=8, min_split_gain=0.0,
        n_estimators=1500, n_jobs=-1, num_leaves=31, objective=None,
        random_state=1, reg_alpha=9, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [38]:
accuracy_score(y_test, clf.predict(X_test))

0.8306511056511057